# Extracting a subset of a table based on values in another

This notebook prepares data for input into a knowledge graph. Below is a description of the two tables:
- **Units** table records a `Unit` and data about the unit (fictitious).
- **UnitRelationships** table records an `OriginUnit` and a `DestinationUnit` and data about the relationship they have (also fictitious).

The **Units** table contains 8 Units. Both the `OriginUnit` and `DestinationUnit` fields in the **UnitRelationships** table use units from the `Unit` table, and other units not in the `Unit` table.

The problem is, we only want to load the data containing information only about the units mentioned in the `Unit` table.

Workflow:
- Load both tables into a pandas dataframe
- Drop rows from **UnitRelationships** table mentioning a unit not in the **Units** table.
- Load data into a knowledge graph using the ArcGIS API for Python. Alternatively, you could export the data frame and use the Load Table wizard in ArcGIS Pro.

## Setting variables and importing libraries

In [ ]:
# For subsetting the table
import pandas as pd

# For editing the knowledge graph
from arcgis.gis import GIS
from arcgis.graph import KnowledgeGraph
import uuid
from pprint import pprint

In [ ]:
url = "https://adsrv2019.ad.local/server/rest/services/Hosted/Units/KnowledgeGraphServer" # URL to knowledge service
gis = GIS("home") # Sign in using ArcGIS Pro's credentials
knowledge_graph = KnowledgeGraph(url=url, gis=gis)
knowledge_graph

In [ ]:
unitsTable = r"C:\DemoData\KNOW\UnitData.gdb\Units"
unitRels = r"C:\DemoData\KNOW\UnitData.gdb\UnitRelationships"

## Create dataframes

In [ ]:
unitsDF = pd.DataFrame.spatial.from_table(unitsTable) 
unitRelsDF = pd.DataFrame.spatial.from_table(unitRels)

This **Unit** table contains the units in the `UnitName` column. These are the only units we are interested in.

In [ ]:
unitsDF

,OBJECTID,UnitName,SomeData
0,1,Unit A,Took training
1,2,Unit B,Lives in Charlseton
2,3,Unit C,Likes banana laffy taffy
3,4,Unit D,Sits in back of room
4,5,Unit E,Helped set up the lab
5,6,Unit F,Unlocked the door
6,7,Unit G,Smart
7,8,Unit H,Also smart


This **UnitRelationship** table is larger, and contains more units than the **Unit** table. We want to filter out the units not in the **Unit** table.

In [ ]:
unitRelsDF

,OBJECTID,OriginUnit,DestinationUnit,MoreData
0,1,Unit A,Unit B,Drinks Celsius
1,2,Unit B,Unit C,Paid for training
2,3,Unit C,Unit B,Attended training
3,4,Unit D,Unit B,Attended training too
4,5,Unit E,Unit B,Attended training as well
5,6,Unit F,Unit C,Childhood friends
6,7,Unit G,Unit H,Sit near each other
7,9,Unit Y,Unit H,Friends
8,10,Unit Q,Unit B,Doesn't like training
9,11,Unit A,Unit Q,Got lost


In [ ]:
unitList = list(unitsDF['UnitName'])
unitList

['Unit A', 'Unit B', 'Unit C', 'Unit D', 'Unit E', 'Unit F', 'Unit G', 'Unit H']

## Dropping record if both values aren't in list

This code uses a pandas dataframe to check if the list of values are used in the `OriginUnit` column and `DestinationUnit` column. If a valid value is in both columns, keep them. Here is a break down of the code:
- `unitRelsDF['OriginUnit'].isin(unitList)` returns a boolean dataframe. `True` indicates the valid value is in the `OriginUnit` column. `False` indicates not.
- `unitRelsDF['DestinationUnit'].isin(unitList)` returns a boolean dataframe. `True` indicates the valid value is in the `DestinationUnit` column. `False` indicates not.
- The `&` requires that the resulting dataframe returns results where both tests are `True`.
- The whole expression returns a dataframe indicating where the condition is met and not met. Including this expression in the square brackets subsets the `unitRelsDF` dataframe. **`unitRelsDF[`**`unitRelsDF['OriginUnit'].isin(unitList) & unitRelsDF['DestinationUnit'].isin(unitList)`**`]`**
- Lastly, set the resulting subsetted dataframe as the `unitRelsDF` variable/s value to overwrite the existing value, which is no longer needed.

In [ ]:
# Including this for the sake of understanding. This code is incorporated in the next code cell.
unitRelsDF['OriginUnit'].isin(unitList)

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7    False
8    False
9     True
Name: OriginUnit, dtype: bool

In [ ]:
unitRelsDF = unitRelsDF[unitRelsDF['OriginUnit'].isin(unitList) & unitRelsDF['DestinationUnit'].isin(unitList)]
unitRelsDF

,OBJECTID,OriginUnit,DestinationUnit,MoreData
0,1,Unit A,Unit B,Drinks Celsius
1,2,Unit B,Unit C,Paid for training
2,3,Unit C,Unit B,Attended training
3,4,Unit D,Unit B,Attended training too
4,5,Unit E,Unit B,Attended training as well
5,6,Unit F,Unit C,Childhood friends
6,7,Unit G,Unit H,Sit near each other


Now that we have the subsetted table, the data can be exported to a file or loaded into a knowledge graph using the ArcGIS API for Python

## Load data in to the knowledge graph with the ArcGIS API for Python

To load the data we will iterate over each row of the subsetted dataframe. Iteratively create the `OriginUnit` enitity, `DestinationUnit` entity, and the relationship connecting them, and apply the edits as each row is processed.
Workflow:
1. Craft a dynamic openCypher query. For each row in the dataframe, check to see if that `UnitName` is already in the knowledge graph. If it is, don't create a new entity, just use it for the relationship. Do this for `OriginUnit` and then the `DestinationUnit`.
2. Rather than just creating an entity for every unit in the dataframe, check to see if it is there. If the entity exists, just grab the globalid (this is in the `if-`**`else`** statement). If the the entity does not exist, create one (this is in the **`if`**`-else` statement). 

In [ ]:
for index, row in unitRelsDF.iterrows():
    # There is no "merge" option in the API for python - you have to handle this yourself. 
    # Before creating entities to use in relationships, check to see if the entity already exists.
    # OriginUnit check
    openCypherQueryOrigin = f"MATCH (u:Unit) WHERE u.UnitName = '{row['OriginUnit']}' RETURN u"    
    qResultOrigin = knowledge_graph.query(openCypherQueryOrigin)
    
    if len(qResultOrigin) == 0:
        # Generate a globalid value using uuid package for the OriginUnit
        oUUID = uuid.uuid4()
    
        # Create the dictionary of properties for the entity relationship representing the OriginUnit
        originEntity = {
            "_objectType": "entity",
            "_typeName": "Unit",
            "_properties": {
                "globalid": oUUID,
                "UnitName": row['OriginUnit']
            }
        }
        knowledge_graph.apply_edits(adds=[originEntity])
    else:
        oUUID = qResultOrigin[0][0]['_properties']['globalid'] # grab the globalid of the existing entity to use in the relationship

    # DestinationUnit check
    openCypherQueryDestination = f"MATCH (u:Unit) WHERE u.UnitName = '{row['DestinationUnit']}' RETURN u"    
    qResultDestination = knowledge_graph.query(openCypherQueryDestination)
    
    if len(qResultDestination) == 0:
        # Generate a globalid value using uuid package for the DestinationUnit
        dUUID = uuid.uuid4()
    
        # Create the dictionary of properties for the entity relationship representing the DestinationUnit
        destinationEntity = {
            "_objectType": "entity",
            "_typeName": "Unit",
            "_properties": {
                "globalid": dUUID,
                "UnitName": row['DestinationUnit']
            }
        }
        knowledge_graph.apply_edits(adds=[destinationEntity])
    else:
        dUUID = qResultDestination[0][0]['_properties']['globalid'] # grab the globalid of the existing entity to use in the relationship
    
    # Create the dictionary of relationship properties to connect the OriginUnit and DestinationUnit in the graph.
    relatedToRelationship = {
        "_objectType": "relationship",
        "_typeName": "RelatedTo",
        "_originEntityId": oUUID,
        "_destinationEntityId": dUUID,
        "_properties": {"MoreData": row['MoreData']
        }
    }
    
    # Apply the edits to the knowledge graph
    knowledge_graph.apply_edits(adds=[relatedToRelationship])
print("Script complete")

Script complete
